In [1]:
import requests
import time
import pandas as pd
import numpy as np
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer

# ПАРСИНГ ВАКАНСИЙ

In [2]:
# url с перечнем специализаций на hh
url = 'https://api.hh.ru/specializations'

In [3]:
# направить запрос и получить специализации
res = requests.get(url)
specializations = res.json()
# создать список id специализаций для обхода ограничения HH в выдаче 2000 items в списке
specialization_ids = []
for i in specializations:
    for j in i['specializations']:
        specialization_ids.append(j['id'])

In [4]:
# создать список id специализаций для обхода ограничения HH в выдаче 2000 items в списке
specialization_ids = []
for i in specializations:
    for j in i['specializations']:
        specialization_ids.append(j['id'])

In [5]:
# параматры для запроса вакансий
sber = '3529'
page = 1
num_per_page = 100

In [6]:
# получить id вакансий для каждой из специализаций по очереди
all_vacancy_ids = []
for i in specialization_ids:
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&specialization={i}'
    res = requests.get(url)
    vacancies = res.json()
    if vacancies.get('found') == 0:
        continue
    for j in range(vacancies.get('pages')):
        url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={j}&per_page={num_per_page}&specialization={i}'
        res = requests.get(url)
        vacancies = res.json()
        vacancy_ids = [el.get('id') for el in vacancies.get('items')]
        all_vacancy_ids.extend(vacancy_ids)
        time.sleep(0.2)

In [11]:
# избавиться от дублей (вакансии в нескольких специализациях)
all_vacancy_ids = list(set(all_vacancy_ids))
len(all_vacancy_ids)

5663

In [12]:
# список ключей, которые запрашивать из данных HH по вакансии
keys = ["id", "name", "area", "salary", "experience", "schedule",
        "employment", "department", "description", "key_skills",
        "specializations", "published_at", "created_at", "alternate_url"]
# Сздать ДФ для наполнения его полученными данными
vacancy_df = pd.DataFrame(columns=keys)

In [13]:
# парсинг данных по найденным id вакансий
for vac_id in all_vacancy_ids:
    url = f'https://api.hh.ru/vacancies/{vac_id}'
    res = requests.get(url)
    vacancy = res.json()
    new_row = {}
    for key in keys:            
        if isinstance(vacancy[key], dict):
            if key == 'salary':
                    new_row[key] = vacancy[key]['from']
            else:
                new_row[key] = vacancy[key]['name']                
        elif vacancy[key] and isinstance(vacancy[key], list):
            arr = []
            for i in vacancy[key]:
                if isinstance(i, dict):
                    arr.append((i['name']))
                else:
                    arr.append(i)
            new_row[key] = arr
        elif vacancy[key]:
            new_row[key] = vacancy[key]
#     добавить инфо по вакансии в ДФ
    vacancy_df = vacancy_df.append(new_row, ignore_index=True)
    time.sleep(0.15)

In [14]:
# Записать полученный ДФ в файл
vacancy_df.to_csv('data_0402.csv')

# Предобработка данных

In [17]:
df = pd.read_csv('data_0402.csv')

In [18]:
# перевести даты создания и публикации в datetime
df['created_at'] = pd.to_datetime(df['created_at'])
df['published_at'] = pd.to_datetime(df['published_at'])

In [19]:
# удалить  тэги из описаний
pattern = r'</*[a-z]+>'
df.description = df.description.apply(lambda x: re.sub(pattern, ' ', x))

# удалить лишние пробелы
pattern = r'\s{1}\s+'
df.description = df.description.apply(lambda x: re.sub(pattern, ' ', x))

# заменить "ё" на "е"
pattern = r'[Ёё]'
df.description = df.description.apply(lambda x: re.sub(pattern, 'е', x))

# заменить ошибочную английскую 'c' в начале слов на русскую 'с'
pattern = r'(c)([а-я]{2,})'
df.description = df.description.apply(lambda x: re.sub(pattern, r'с\2', x))
# заменить ошибочную русскую 'с' на английскую 'c' в начале слов 
pattern = r'(с)([a-z]{2,})'
df.description = df.description.apply(lambda x: re.sub(pattern, r'c\2', x))

# исключить вакансии на английском
pattern = r'[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+[^А-Яа-я]+\s+[^А-Яа-я]'
for i in df.index.values:
    eng = re.match(pattern, df.loc[i, 'description'])
    if eng is not None:
        df.drop(i, inplace=True)
len(df)

5646

## Нормализация описаний вакансий

- Очистка от знаков препинания
- Избавление от словоформ

In [20]:
# функция удаления знаков пунктуации
def punc_clean(string):
    '''Функция ощищает текст от знаков пунктуации'''
    pattern = r'[^А-Яа-яA-Za-z0-9+#]+'
    try:
      string = re.sub(pattern, ' ', string)
    except Exception as e:
      print(e)
    return string

In [21]:
morph = pymorphy2.MorphAnalyzer()

# функция лемматизации
def lemmatize(data, col=None):
    '''Функция приведения слов к начальной форме'''
    t = []
    if type(data) == pd.core.series.Series:
        text = data[col]
    else:
        text = data
    for word in text.split():
        if len(word)<=3:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [22]:
def preprocess(desription):
    desription = punc_clean(desription)
    desription = lemmatize(desription)
    return desription

In [23]:
df['desc_normal'] = df.description.apply(preprocess)

Создание списка стоп-слов

In [25]:
# стоп слова
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# расширение списка стоп-слов
stop_words_ru = stopwords.words('russian')
stop_words_ru.extend(['вакансия', 'каждый', 'который', 'команда', 'любой', 'наш', 'самый', 'сбер', 'сбербанк', 'сберпрайм', 'сберпрайм+', 'сбербизнес', 'свой', 'экосистема', 'подразделение', 'интересный', 'работодатель', 'хороший', 'откликнуться', 'детский', 'родитель', 'предлагать', 'профессиональный', 'библиотека', 'жизнь', 'гибкий', 'график', 'оптимальный', 'баланс', 'работа', 'личный', 'жизнь', 'конкурентный', 'компенсация', 'оклад', 'премия', 'результат', 'деятельность', 'труд', 'белый', 'заработный', 'плата', 'стандартный', 'возможно', 'смещение', 'ранний', 'поздний', 'медицинский', 'страховка', 'родственник', 'комфортный', 'офис', 'столовая', 'широкий', 'спектр', 'дисконт', 'привилегия', 'эффективный', 'трудоустройство', 'согласно', 'гарантировать', 'доход', 'плюс', 'премиальный', 'работать', 'рядом', 'дом', 'место', 'учёба', 'регулярный', 'несчастный', 'случай', 'тяжёлый', 'заболевание', 'помощь', 'социальный', 'кредитование', 'яркий', 'насыщенный', 'ждать', 'строить', 'счастливый', 'будущее', 'вместе', 'мечтать', 'дружный', 'начать', 'карьера', 'крупный', 'банк', 'россия', 'кредитный', 'машина', 'ключевой', 'предлагать', 'современный', 'вблизи', 'москва', 'сити', 'фитнес', 'залом', 'бесплатный', 'подземный', 'паркинг', 'минута', 'метро', 'позитивный', 'зарядить'])
string = 'подготовить ты ипотека выгодный каждый сотрудник льготный условие первый льготный страхование близкий бесплатный подписка сберпрайм+ скидка продукт компания партнёр okko сбер маркет delivery club самокат ситимобила сбер еаптека другой обучение счёт компания онлайн курс виртуальный школа возможность получить новый квалификация курс будущий родитель материальный поддержка молодой детский отдых подарок счёт компания реферальный программа сотрудник можно пригласить команда знакомый профессионал получить вознаграждение рубль корпоративный пенсионный программа интересный вакансия откликнуться записать видеоинтервью ссылка письмо любой удобный устройство пройти собеседование'
string = string.split()
stop_words_ru.extend(string)
stop_words_ru = list(set(stop_words_ru))

In [27]:
# описание опыта из резюме
expirience = '''Консультирование и продажа банковских продуктов: открытие вкладов, кредитов, кредитных и дебетовых карт, \
подключение автоплатежей, продажа инвестиционных продуктов, переводы и т.д.'''

Предобработка опыта

In [28]:
expirience = preprocess(expirience)

## 2. ТОКЕНИЗАЦИЯ И ВЕКТОРИЗАЦИЯ

### МЕТОД 1. Граф векторизатор

In [29]:
vec_CV = CountVectorizer(stop_words=stop_words_ru, min_df=0.002)

In [30]:
desc_vectors_CV = vec_CV.fit_transform(df['desc_normal'])

In [31]:
print('Vocabulary length:', len(vec_CV.get_feature_names()))

Vocabulary length: 3230


Добавление в DF векторных представлений описаний

In [32]:
df['CV_vector'] = list(desc_vectors_CV.toarray())

Векторизация опыта из резюме

In [33]:
# для токенизации применяем словарь эмбеддинга
vec_exp = CountVectorizer(stop_words=stop_words_ru, vocabulary=vec_CV.get_feature_names())
exp_vector_CV = vec_exp.fit_transform([expirience])

Определение косинусного сходства опыта с описаниями вакансий

In [34]:
# функция для определения косинусного сходства
def cosine_sim_vectors(vec1, vec2):
    '''Функция для определения косинусной меры сходства'''
    return cosine_similarity([vec1, vec2])[1][0]

In [35]:
df['CV_cosine'] = df['CV_vector'].apply(cosine_sim_vectors, vec2=exp_vector_CV.toarray()[0])

In [ ]:
top_10_CV = df.sort_values(by='CV_cosine', ascending=False).head(10)
top_10_CV[["name", "description", "specializations"]]

In [36]:
top_10_CV = df.sort_values(by='CV_cosine', ascending=False).head(10)
top_10_CV[["name", "description", "specializations"]]

,name,description,specializations
3941,Консультант по работе с пенсионным фондом,Обязанности: Открытие (продажа) банковских ка...,"['Страхование жизни', 'Автострахование', 'Пере..."
3330,Инвестиционный менеджер (канал Сбербанк Первый),Тебе предстоит: осуществление кураторства зак...,"['Инвестиционная компания', 'Прямые инвестиции..."
4810,Региональный менеджер внешней дистрибуции,Обязанности: организация процесса продаж для ...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
986,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
757,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
5504,Менеджер по продажам продуктов ВЭД,Продажи продуктов ВЭД Обязанности : продажа п...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
5418,Менеджер поддержки продаж,Обязанности: консультирование по банковским п...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."
3963,Финансовый советник,привлечение состоятельных Клиентов в канал Сб...,"['Методология, Банковские технологии', 'Разраб..."
5055,Консультант по работе с ПФР,Обязанности: Проведение презентаций по продаж...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."
3739,Консультант по работе с ПФР,Обязанности: Проведение презентаций по продаж...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."


### МЕТОД 2. TfidfVectorizer

In [37]:
vec_tfidf = TfidfVectorizer(stop_words=stop_words_ru, min_df=0.002)

In [38]:
desc_vectors_tfidf = vec_tfidf.fit_transform(df['desc_normal'])

In [39]:
df['TfIdf_vector'] = list(desc_vectors_tfidf.toarray())

In [41]:
# токенизация описания опыта
vec_exp_tfidf = TfidfVectorizer(stop_words=stop_words_ru, vocabulary=vec_tfidf.get_feature_names())
# токенизация описания опыта
exp_vector_tfidf = vec_exp_tfidf.fit_transform([expirience])

In [42]:
df['TfIdf_cosine'] = df['TfIdf_vector'].apply(cosine_sim_vectors, vec2=exp_vector_tfidf.toarray()[0])

In [43]:
top_10_TV = df.sort_values(by='TfIdf_cosine', ascending=False)[["name", "description", "specializations"]].head(10)
top_10_TV

,name,description,specializations
3941,Консультант по работе с пенсионным фондом,Обязанности: Открытие (продажа) банковских ка...,"['Страхование жизни', 'Автострахование', 'Пере..."
4810,Региональный менеджер внешней дистрибуции,Обязанности: организация процесса продаж для ...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
757,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
986,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
5418,Менеджер поддержки продаж,Обязанности: консультирование по банковским п...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."
5504,Менеджер по продажам продуктов ВЭД,Продажи продуктов ВЭД Обязанности : продажа п...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
3330,Инвестиционный менеджер (канал Сбербанк Первый),Тебе предстоит: осуществление кураторства зак...,"['Инвестиционная компания', 'Прямые инвестиции..."
3984,Консультант по работе с Пенсионным Фондом России,Обязанности: организация качественного провед...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
5055,Консультант по работе с ПФР,Обязанности: Проведение презентаций по продаж...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."
3739,Консультант по работе с ПФР,Обязанности: Проведение презентаций по продаж...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."


In [139]:
top_10_TV

,name,description,specializations
3928,Консультант по работе с пенсионным фондом,Обязанности: Открытие (продажа) банковских ка...,"['Страхование жизни', 'Автострахование', 'Пере..."
4795,Региональный менеджер внешней дистрибуции,Обязанности: организация процесса продаж для ...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
754,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
982,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
5402,Менеджер поддержки продаж,Обязанности: консультирование по банковским п...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."
5487,Менеджер по продажам продуктов ВЭД,Продажи продуктов ВЭД Обязанности : продажа п...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
3318,Инвестиционный менеджер (канал Сбербанк Первый),Тебе предстоит: осуществление кураторства зак...,"['Инвестиционная компания', 'Прямые инвестиции..."
3971,Консультант по работе с Пенсионным Фондом России,Обязанности: организация качественного провед...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
5040,Консультант по работе с ПФР,Обязанности: Проведение презентаций по продаж...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."
3727,Консультант по работе с ПФР,Обязанности: Проведение презентаций по продаж...,"['Привлечение клиентов', 'Кредиты', 'Продажа ф..."


### МЕТОД 3. Doc2Vec

In [46]:
# функция для удаления стоп-слов (в методе нет встроенного параметра)
def remove_stopwords(document, stopwords=stop_words_ru, return_string=False):
    '''Функция для удаления стоп-слов'''
    document = [word for word in document if word not in stopwords]
    if return_string:
        document = " ".join(document)
    return document

In [48]:
tagged_data = [TaggedDocument(words=remove_stopwords(word_tokenize(text)),
                              tags=[i]) for i, text in zip(list(df.desc_normal.index.values), df.desc_normal.values)]

In [49]:
doc2vec_model = Doc2Vec(vector_size=100, 
                min_alpha=0.00025,
                min_count=int(len(df)*0.001),
                dm=0,
                epochs=20)

doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data,
            total_examples=doc2vec_model.corpus_count,
            epochs=doc2vec_model.epochs)

In [50]:
# векторизация опыта из резюме на основании полученного эмбеддинга

exp_tokens = remove_stopwords(word_tokenize(expirience))
exp_vector_doc2vec = doc2vec_model.infer_vector(exp_tokens)

In [52]:
# подбор наиболее подходящих вакансий
# similar_vac = doc2vec_model.docvecs.most_similar(exp_vector_doc2vec, topn=10)
# for i in similar_vac:
#     print(i[0], df.name.loc[i[0]])

In [53]:
df['D2V_vector'] = [doc2vec_model.dv[i] for i in df.index.values]

In [54]:
df['D2V_cosine'] = df['D2V_vector'].apply(cosine_sim_vectors, vec2=exp_vector_doc2vec)

In [55]:
top_10_D2V = df.sort_values(by='D2V_cosine', ascending=False)[["name", "description", "specializations"]].head(10)
top_10_D2V

,name,description,specializations
3941,Консультант по работе с пенсионным фондом,Обязанности: Открытие (продажа) банковских ка...,"['Страхование жизни', 'Автострахование', 'Пере..."
5023,Клиентский менеджер Сбер Премьер в МИДДЛ-офис,Обязанности: - обслуживание премиальных клиен...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
288,Клиентский менеджер Сбер Премьер в МИДДЛ-офис,Обязанности: - обслуживание премиальных клиен...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
4017,Менеджер по продаже жилищных кредитов,консультация клиентов и партнеров банка по пр...,"['Привлечение клиентов', 'Продажа финансовых п..."
2851,Клиентский менеджер внешней дистрибуции,"Хочешь работать в банке, но офисная рутина не...","['Прямые продажи', 'Оптовая торговля', 'Менедж..."
5138,Менеджер по работе с клиентами малого бизнеса,Обязанности: консультировать корпоративных кл...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
5487,Клиентский менеджер внешней дистрибуции,"Хочешь работать в банке, но офисная рутина не...","['Прямые продажи', 'Оптовая торговля', 'Менедж..."
2168,Руководитель по внедрению проектов (КМ-хантер),Обязанности: формирование потребности и разви...,"['Информационные технологии', 'Управление прое..."
777,Консультант по работе с Пенсионным Фондом России,Обязанности: - организация качественного пров...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
3984,Консультант по работе с Пенсионным Фондом России,Обязанности: организация качественного провед...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."


### МЕТОД 4. SentenceTransformer (BERT)

In [57]:
ST_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [58]:
df['bert_vector'] = df.desc_normal.apply(ST_model.encode)

In [59]:
exp_vector_BERT = ST_model.encode(expirience)

In [60]:
df['bert_cosine'] = df['bert_vector'].apply(cosine_sim_vectors, vec2=exp_vector_BERT)

In [61]:
top_10_BERT = df.sort_values(by='bert_cosine', ascending=False).head(10)[['id', 'name']]

In [62]:
top_10_BERT

,id,name
5023,51860457.0,Клиентский менеджер Сбер Премьер в МИДДЛ-офис
288,51978275.0,Клиентский менеджер Сбер Премьер в МИДДЛ-офис
5418,50979215.0,Менеджер поддержки продаж
777,51405259.0,Консультант по работе с Пенсионным Фондом России
3984,51417228.0,Консультант по работе с Пенсионным Фондом России
3739,50006422.0,Консультант по работе с ПФР
5055,48031761.0,Консультант по работе с ПФР
4268,51311334.0,Клиентский менеджер внешней дистрибуции (Прямы...
5138,51390286.0,Менеджер по работе с клиентами малого бизнеса
3556,49749744.0,"Финансовый эксперт, г. Ставрополь"


# Скрипт подбора вакансий

In [63]:
# Клиентский менеджер
experience_ = '''Консультирование и продажа банковских продуктов: открытие вкладов, кредитов, кредитных и дебетовых карт, \
подключение автоплатежей, продажа инвестиционных продуктов, переводы'''

In [66]:
methods = {
    'CountVectorizer': {'model': vec_CV, 'embedding': desc_vectors_CV},
    'TfidfVectorizer': {'model': vec_tfidf, 'embedding': desc_vectors_tfidf},
    'Doc2Vec': {'model': doc2vec_model},
    'SentenceTransformer': {'model': ST_model}
}

In [67]:
def cv_vectorizer(experience_description: str, sw=stop_words_ru, voc=vec_CV.get_feature_names()):
    vectorizer = CountVectorizer(stop_words=sw, vocabulary=voc)
    exp_vector = vectorizer.fit_transform([experience_description])
    exp_vector = exp_vector.toarray()[0]
    return exp_vector

In [68]:
def tfidf_vectorizer(experience_description, sw=stop_words_ru, voc=vec_tfidf.get_feature_names()):
    vectorizer = TfidfVectorizer(stop_words=sw, vocabulary=voc)
    exp_vector = vectorizer.fit_transform([experience_description])
    exp_vector = exp_vector.toarray()[0]
    return exp_vector

In [69]:
def d2v_vectorizer(experience_description, sw=stop_words_ru, model=doc2vec_model):
    exp_tokens = [word for word in experience_description.split() if word not in sw]
    exp_vector = model.infer_vector(exp_tokens)
    return exp_vector

In [70]:
def st_vectorizer(experience_description, model=ST_model):
    exp_vector = model.encode(expirience)
    return exp_vector

In [71]:
def vacancy_recommender(experience_description: str, method: str):
    '''
    Parameters:
        experience_description: str
            Текстовое описание опыта из резюме.
        method: str
            Применяемый метод векторизации.
            Возможные значения: 'CountVectorizer',
                                'TfidfVectorizer',
                                'Doc2Vec',
                                'SentenceTransformer'
    '''
    methods = {
        'CountVectorizer': {'df_col': 'CV_vector', 'vectorizer': cv_vectorizer},
        'TfidfVectorizer': {'df_col': 'TfIdf_vector', 'vectorizer': tfidf_vectorizer},
        'Doc2Vec': {'df_col': 'D2V_vector', 'vectorizer': d2v_vectorizer},
        'SentenceTransformer': {'df_col': 'bert_vector', 'vectorizer': st_vectorizer}
    }
    exp_normal = preprocess(experience_description)
    exp_vector = methods[method]['vectorizer'](exp_normal)
    tmp_df = df[['id', 'name', 'description', 'specializations', methods[method]['df_col']]].copy()
    tmp_df['cosine'] = tmp_df[methods[method]['df_col']].apply(cosine_sim_vectors, vec2=exp_vector)
    top_10 = tmp_df.sort_values(by='cosine', ascending=False)[['name', 'description']].head(10)
    return top_10
    

In [72]:
vacancy_recommender(experience_, 'Doc2Vec')

,name,description
3941,Консультант по работе с пенсионным фондом,Обязанности: Открытие (продажа) банковских ка...
5023,Клиентский менеджер Сбер Премьер в МИДДЛ-офис,Обязанности: - обслуживание премиальных клиен...
288,Клиентский менеджер Сбер Премьер в МИДДЛ-офис,Обязанности: - обслуживание премиальных клиен...
4017,Менеджер по продаже жилищных кредитов,консультация клиентов и партнеров банка по пр...
2851,Клиентский менеджер внешней дистрибуции,"Хочешь работать в банке, но офисная рутина не..."
777,Консультант по работе с Пенсионным Фондом России,Обязанности: - организация качественного пров...
1343,Консультант по работе с ПФР,Обязанности: обеспечение выполнения плана по ...
5487,Клиентский менеджер внешней дистрибуции,"Хочешь работать в банке, но офисная рутина не..."
3984,Консультант по работе с Пенсионным Фондом России,Обязанности: организация качественного провед...
5138,Менеджер по работе с клиентами малого бизнеса,Обязанности: консультировать корпоративных кл...


In [73]:
# Java-разработчик
experience_ = '''Java-разработчик
-Разработка прикладного программного обеспечения по реализации продуктов в рамках Зарплатного Проекта Сбербанка (Java 11, Spring Boot, Postgres)
-Разработка по Agile, коммуникация с менеджерами проектов, дизайнерами, аналитиками и тестировщиками.
Java Developer
-Разработка серверной части веб приложений для программно-аппаратных комплексов (IoT)
-Проектирование api для взаимодействия бекенд-фронтенд
-Разработка и реализация REST API с использованием Spring, Spring Boot, Security
-Участие в проектировании структуры БД (PostgreSQL)
Инженер-разработчик
Разработка веб-приложения на Java (full-stack) под нужды команды нагрузочного тестирования:
-Проектирование и реализация БД (sqlite, mysql, jdbc, hybernate)
-Разработка и реализация серверной части приложения на java (Spring, Boot, Security)
-Разработка и реализация фронт части (html, css, js, thymeleaf, bootstrap)
-Разработка REST API и написание документации
-Покрытие тестами (spring test, junit, mockito)
-Разработка пайплайнов (gitlab ci/cd)'''

In [76]:
vacancy_recommender(experience_, 'TfidfVectorizer')

,name,description
73,Java-разработчик,О проекте: Мы занимаемся разработкой сервиса ...
4938,Java-разработчик,Наша команда Каналы и поверхности - Поставка ...
1693,Java разработчик,"Сбер сейчас это, крупнейшая цифровая экосисте..."
2984,Java Developer (Fair Pricing),Мы приглашаем в нашу команду сильных амбициоз...
5298,Java developer,Предметная область. #SberData – департамент п...
4372,Главный инженер по разработке Java,- Проектирование и разработка пользовательско...
3944,Java разработчик (Каналы и поверхности),Обязанности: Непосредственное участие во всех...
4046,Java-разработчик (проект Fair Pricing),Мы приглашаем в нашу команду сильных амбициоз...
1318,Java-разработчик,Мы приглашаем в нашу команду сильных амбициоз...
1548,Java-разработчик (проект Fair Pricing),Мы приглашаем в нашу команду сильных амбициоз...


In [77]:
vacancy_recommender(experience_, 'Doc2Vec')

,name,description
3183,Java-разработчик (проект Платежи и переводы),Проект: Мы - Кластер «Payments&amp;Transfers»...
1998,"Java Developer (Проект ""Кредитные карты"")",Описание проекта Мы разрабатываем принципиаль...
4024,PHP-программист,Обязанности: - проектирование и разработка но...
4020,Java разработчик (Эмиссии зарплатных карт),Ищем Java-разработчика в Зарплатный проект Ба...
2124,Java developer (Виртуальный ассистент КБ),"Мы ищем разработчика JAVA в ПАО Сбербанк, нео..."
4605,Java-разработчик,Мы – команда блока Sberbank International. Мы...
1279,Team Lead / Senior Java-разработчик,Обязанности: Команда разработки платформенных...
2947,Специалист Java,Экспертный уровень знаний Java Знание Spring ...
1378,Android-разработчик,Мы - команда разработчиков из Экосистемы Сбер...
3537,System Analyst / Software Architect,В команду профессионалов требуется технически...
